In [28]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from tqdm import tqdm

Let's fetch the dataframes.

In [39]:
model = SentenceTransformer('all-mpnet-base-v2')
vtt_domain_df = pd.read_csv("./data/dataframes/df_relationships_vtt_domain.csv")
comp_domain_df = pd.read_csv("./data/dataframes/df_relationships_comp_domain.csv")

In [43]:
comp_domain_df.head(2)

,Document number,Source Company,relationship description,source id,source type,source description,relationship type,target id,target type,target description,Link Source Text,Source Text
0,0,FORTUM OYJ,Fortum Corporation is responsible for executin...,First nuclear decommissioning project in Finla...,Innovation,The first nuclear reactor decommissioning proj...,DEVELOPED_BY,Fortum Corporation,Organization,A company with over 40 years of experience in ...,https://www.fortum.com/media/2020/04/fortum-aw...,"Press release\n 08 April 2020, 7:00 EEST ..."
1,0,FORTUM OYJ,VTT Technical Research Centre of Finland Ltd c...,First nuclear decommissioning project in Finla...,Innovation,The first nuclear reactor decommissioning proj...,DEVELOPED_BY,VTT Technical Research Centre of Finland Ltd,Organization,A research organization in Finland contracting...,https://www.fortum.com/media/2020/04/fortum-aw...,"Press release\n 08 April 2020, 7:00 EEST ..."


In [42]:
vtt_domain_df.head(2)

,Document number,VAT id,relationship description,source id,source type,source english_id,source description,relationship type,target id,target type,target english_id,target description,Link Source Text,Source Text
0,0,FI10292588,"FiR 1 nuclear research reactor was developed, ...",FiR 1,Innovation,FiR 1,FiR 1 is a Triga-type nuclear research reactor...,DEVELOPED_BY,FI26473754,Organization,VTT Technical Research Centre of Finland Ltd.,VTT is a Finnish research and innovation partn...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...
1,0,FI10292588,Centre for Nuclear Safety is being developed a...,Centre for Nuclear Safety,Innovation,Centre for Nuclear Safety,A modern research facility under construction ...,DEVELOPED_BY,FI26473754,Organization,VTT Technical Research Centre of Finland Ltd.,VTT is a Finnish research and innovation partn...,https://www.vttresearch.com/en/news-and-ideas/...,Skip to main content Beyond the obvious Open m...


In [ ]:
def create_text_to_compare(row):
    text_to_compare = row["source_id"]

In [ ]:
# Clean the VTT domain df.



vtt_domain_df["text_to_compare"] = (
    vtt_domain_df["source id"].fillna("") + " " +
    vtt_domain_df["source english_id"].fillna("") + " " +
    vtt_domain_df["source description"].fillna(""))

vtt_domain_df = vtt_domain_df[vtt_domain_df["source type"] != "Organization"]
vtt_domain_df = vtt_domain_df.drop_duplicates(subset="source description", keep="first")

# Clean the comp domain df.

comp_domain_df["text_to_compare"] = (
    comp_domain_df["source id"].fillna("") + " " +
    comp_domain_df["source description"].fillna(""))

comp_domain_df = comp_domain_df[comp_domain_df["source type"] != "Organization"]
comp_domain_df = comp_domain_df.drop_duplicates(subset="source description", keep="first")

print("Comp domain df length:", len(comp_domain_df), " - VTT domain df length:", len(vtt_domain_df))

Comp domain df length: 1528  - VTT domain df length: 1504


Embed everything.

In [36]:
vtt_embeddings = model.encode(vtt_domain_df["text_to_compare"].tolist(), convert_to_tensor=True)
comp_embeddings = model.encode(comp_domain_df["text_to_compare"].tolist(), convert_to_tensor=True)

In [37]:
threshold = 0.80
similar_pairs = []

print("Comparing entries pair by pair...")

for i in tqdm(range(len(vtt_embeddings))):
    for j in range(i + 1, len(comp_embeddings)):
        sim = util.cos_sim(vtt_embeddings[i], comp_embeddings[j]).item()
        
        if sim > threshold:
            source_i = vtt_domain_df.iloc[i]["text_to_compare"]
            source_j = comp_domain_df.iloc[j]["text_to_compare"]
            
            print(f"\nSimilarity: {sim:.3f} | From: {i+1} To: {j+1}")
            print(f"→ Source {i+1} description: {source_i}")
            print(f"→ Source {j+1} description: {source_j}")
            
            similar_pairs.append((i, j, sim))

# Output results
print(f"\nFound {len(similar_pairs)} high-similarity pairs (> {threshold}):")
for i, j, score in similar_pairs:
    print(f"[{i}] '{df.iloc[i]['source english_id']}' <--> [{j}] '{df.iloc[j]['source english_id']}' | Similarity: {score:.3f}")

Comparing entries pair by pair...


  0%|          | 0/1504 [00:00<?, ?it/s]


Similarity: 0.828 | From: 1 To: 880
→ Source 1 description: FiR 1 FiR 1 FiR 1 is a Triga-type nuclear research reactor located in Otaniemi, Espoo, Finland, decommissioned by VTT after serving since 1962, used for nuclear research, training, and medical radiation therapy.
→ Source 880 description: FiR 1 -tutkimusreaktori A 250 kilowatt research nuclear reactor located in Otaniemi, Espoo, used from 1962 to 2015, now undergoing decommissioning.


  0%|          | 1/1504 [00:00<21:03,  1.19it/s]


Similarity: 0.803 | From: 2 To: 285
→ Source 2 description: Centre for Nuclear Safety Centre for Nuclear Safety A modern research facility under construction by VTT in Otaniemi, Espoo for nuclear safety research, equipped with state-of-the-art technology for studying radioactive materials and improving nuclear safety.
→ Source 285 description: nuclear safety house A new four-storey building bringing together all nuclear safety research activities of VTT under one roof, integrating office and laboratory spaces with an activity-based environment to improve collaboration and experimental research capabilities.


...


Similarity: 0.838 | From: 74 To: 1506
→ Source 74 description: BioCelSol technology BioCelSol technology A technology that enhances cellulose dissolution via mechanical and enzymatic treatments prior to fibre spinning.
→ Source 1506 description: BioCelSol-teknologia A technology demonstrated by VTT that enhances cellulose dissolution via mechanical-enzymatic treatment before solvent and spinning processes for textile fiber production.


  5%|▍         | 75/1504 [00:48<15:59,  1.49it/s]


Similarity: 0.872 | From: 76 To: 991
→ Source 76 description: Finnish 5G technology enables efficient mobile streaming to large audiences Finnish 5G technology enables efficient mobile streaming to large audiences A novel 5G technology in Finland using evolved Multimedia Broadcast Multicast Service (eMBMS) enabling high-quality live video streaming to large audiences on commercial smartphones in test networks.
→ Source 991 description: evolved Multimedia Broadcast Multicast Service (eMBMS) technology A technology enabling live streaming of TV channels in mobile wireless networks, enhancing bandwidth efficiency and reliability, demonstrated for the first time in Finland in 5G test networks.


  5%|▌         | 77/1504 [00:50<15:28,  1.54it/s]


KeyboardInterrupt: 